In [ ]:
import gudhi
import pickle
import numpy as np
import gudhi as gd
from LISM_simplicial import *


## Load data

In [ ]:
with open('activation_data_mnist/simplexTree.pickle', 'rb') as handle:
    simplex_list = pickle.load(handle)

ST = gd.SimplexTree()
initial_value = -1e10

for simplex in simplex_list:
    ST.insert(simplex, initial_value)


convention label : 0 normal, 1 fgsm, 2 fgsm noise, 3 cw, 4 cw noise

In [ ]:
fgsm = np.load('activation_data_mnist/fgsm.npy')
fgsm_noise = np.load('activation_data_mnist/fgsm_noise.npy')
normal = np.load('activation_data_mnist/normal.npy')
cw = np.load('activation_data_mnist/cw.npy')
cw_noise = np.load('activation_data_mnist/cw_noise.npy')

## Compute distance matrix with possible data leakage

In [ ]:
n_inputs = 20
features = [0,1]
data = np.zeros((n_inputs * 5, 260, 2))
labels = []
for i in range(5):
    labels += [i]*n_inputs


In [ ]:
data[0:n_inputs, : ,: ] = fgsm[0:n_inputs, : , features ]
data[n_inputs:2*n_inputs, : ,: ] = fgsm_noise[0:n_inputs, : , features ]
data[2*n_inputs:3*n_inputs, : ,: ] = normal[0:n_inputs, : , features ]
data[3*n_inputs:4*n_inputs, : ,: ] = cw[0:n_inputs, : , features ]
data[4*n_inputs:5*n_inputs, : ,: ] = cw_noise[0:n_inputs, : , features ]


In [ ]:
data.shape

In [ ]:
distance_matrix = adversarial_pipeline(simplex_list, data)